# Private RAG Application Implementation Using Llama 3.2, Ollama & PostgreSQL

Install Psycopg

```bash
pip install "psycopg[binary,pool]"
```

In [2]:
import psycopg

In [ ]:
DATABASE_HOST = ''  # IP address for Postgres database
OLLAMA_HOST = ''  # Location of Ollama (http://host.docker.internal:11434 if using already existing ollama instance)

## Dataset Preparation

In [56]:
# the knowledge base
dummy_data = [
    {"title": "Seoul Tower", "content": "Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea."},
    {"title": "Gwanghwamun Gate", "content": "Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea."},
    {"title": "Bukchon Hanok Village", "content": "Bukchon Hanok Village is a Korean traditional village in Seoul with a long history."},
    {"title": "Myeong-dong Shopping Street", "content": "Myeong-dong is one of the primary shopping districts in Seoul, South Korea."},
    {"title": "Dongdaemun Design Plaza", "content": "The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea."}
]

## Connect to the database

In [4]:
def connect_db():
    return psycopg.connect( # use the credentials of your postgresql database 
        host = DATABASE_HOST,
        dbname = 'postgres',
        user = 'postgres',
        password = 'password',
        port = '5432'
    )

## Create the table

In [58]:
conn = connect_db()
cur = conn.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            title TEXT,
            content TEXT,
            embedding VECTOR(768)
        );
    """)
conn.commit()
cur.close()
conn.close()

### Insert the data and embeddings into the database

In [ ]:
conn = connect_db()
cur = conn.cursor()

# Use the port at which your Ollama service is running.
for doc in dummy_data:
    # Prepare the concatenated string in Python
    concatenated_value = f"{doc['title']} - {doc['content']}"
    
    # Use the concatenated value directly in the SQL query with positional placeholders
    cur.execute("""
        INSERT INTO documents (title, content, embedding)
        VALUES (
            %s,
            %s,
            ai.ollama_embed('nomic-embed-text', %s, host=>%s)
        )
    """, (doc['title'], doc['content'], concatenated_value, OLLAMA_HOST))

conn.commit()
cur.close()
conn.close()



Safechecking the insert information in the database.

In [5]:
conn = connect_db()
cur = conn.cursor()
    
cur.execute("""
    SELECT title, content, vector_dims(embedding) 
    FROM documents;
""")

rows = cur.fetchall()
for row in rows:
    print(f"Title: {row[0]}, Content: {row[1]}, Embedding Dimensions: {row[2]}")

cur.close()
conn.close()

Title: Seoul Tower, Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea., Embedding Dimensions: 768
Title: Gwanghwamun Gate, Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea., Embedding Dimensions: 768
Title: Bukchon Hanok Village, Content: Bukchon Hanok Village is a Korean traditional village in Seoul with a long history., Embedding Dimensions: 768
Title: Myeong-dong Shopping Street, Content: Myeong-dong is one of the primary shopping districts in Seoul, South Korea., Embedding Dimensions: 768
Title: Dongdaemun Design Plaza, Content: The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea., Embedding Dimensions: 768


## Retrieval and Generation

### Define the query and its embedding

In [36]:
query = "Tell me about gates in South Korea."

In [ ]:
conn = connect_db()
cur = conn.cursor()
    
# Embed the query using the ollama_embed function
cur.execute("""
    SELECT ai.ollama_embed('nomic-embed-text', %s, host=>%s);
""", (query,OLLAMA_HOST))
query_embedding = cur.fetchone()[0]

# Retrieve relevant documents based on cosine distance
cur.execute("""
    SELECT title, content, 1 - (embedding <=> %s) AS similarity
    FROM documents
    ORDER BY similarity DESC
    LIMIT 3;
""", (query_embedding,))

rows = cur.fetchall()
    
# Prepare the context for generating the response
context = "\n\n".join([f"Title: {row[0]}\nContent: {row[1]}" for row in rows])
print(context)

cur.close()
conn.close()

Title: Gwanghwamun Gate
Content: Gwanghwamun is the main and largest gate of Gyeongbokgung Palace, in Jongno-gu, Seoul, South Korea.

Title: Dongdaemun Design Plaza
Content: The Dongdaemun Design Plaza is a major urban development landmark in Seoul, South Korea.

Title: Seoul Tower
Content: Seoul Tower is a communication and observation tower located on Namsan Mountain in central Seoul, South Korea.


In [ ]:
conn = connect_db()
cur = conn.cursor()

# Generate the response using the ollama_generate function
cur.execute("""
    SELECT ai.ollama_generate('llama3.2', %s, host=>%s);
""", (f"Query: {query}\nContext: {context}",OLLAMA_HOST))
    
model_response = cur.fetchone()[0]
print(model_response['response'])
    
cur.close()
conn.close()

Gates in South Korea hold significant cultural and historical importance, particularly in the context of traditional palaces and monuments. Here's an overview of some notable gates in South Korea:

1. **Gwanghwamun Gate**: Located within Gyeongbokgung Palace, Gwanghwamun is the main and largest gate of the palace complex in Jongno-gu, Seoul. The gate was built during the reign of King Jeonjo (1724-1776) and is a symbol of traditional Korean architecture.
2. **Dongdaemun Gate**: Originally built as an entrance to Gyeongbokgung Palace, Dongdaemun Gate was renovated in 2011 to serve as the main gate of the Dongdaemun Design Plaza (DDP). The DDP is a major urban development landmark and a hub for fashion, design, and culture in Seoul.
3. **Seoraksan Gate**: Located at the entrance of Seoraksan National Park, this gate marks the beginning of a popular hiking trail that leads to the park's scenic trails and stunning views.

Gates in South Korea often serve as gateways to historical sites, cu